In [1]:
from __future__ import division
from pyomo.opt import SolverFactory
from pyomo.environ import *
import time
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from scipy.linalg import norm
import seaborn as sns
from pyomo.opt import SolverStatus, TerminationCondition

In [2]:
def Co_optimization(inputfile, efficiency, solver="gurobi"):
    # Read all price data

    price = inputfile['price'].to_dict()
    netload = inputfile['Demand (kW)'].to_dict()


    with SolverFactory(solver) as opt:
        # Creation of a Concrete Model
            model = ConcreteModel()


            # ###### Set
            model.t = Set(initialize=range(96))

            # ###### Parameters

            # Price
            model.price_energy = Param(model.t, initialize=price, doc='Energy Price')
            model.net_load = Param(model.t, initialize = netload, doc = 'net load')


            # ###### Variable
            model.energy_charge = Var(model.t, domain=NonNegativeReals, doc='charge power')
            model.energy_discharge = Var(model.t, domain=NonNegativeReals, doc='discharge power')
            #model.SOE = Var(model.t, domain=NonNegativeReals, initialize= inisoc, doc='charge power')
            model.V = Var(domain = NonNegativeReals, doc = 'added demand charge cost')

            # define 0-1 variable
            model.u = Var(model.t, within=Binary, doc='positive energy sign')
            model.v = Var(model.t, within=Binary, doc='negative energy sign')


            # ###### Rules
            def maximum_chargepower_rule(model, t):
                return model.energy_charge[t]<= model.u[t] * 400
            model.chargepower_max_rule1 = Constraint(model.t, rule=maximum_chargepower_rule, doc='Pcharge max rule')

            def maximum_dischargepower_rule(model, t):
                return model.energy_discharge[t]<= model.v[t] * 400
            model.dischargepower_max_rule2 = Constraint(model.t, rule=maximum_dischargepower_rule, doc='Pdischarge max rule')


            def uv_rule(model,t):
                return model.u[t] + model.v[t] <= 1
            model.uvtime = Constraint(model.t, rule=uv_rule, doc='not charge and discharge simultanously')


            def minimum_energy_rule(model, t):
                return sum(model.energy_charge[i]*efficiency - model.energy_discharge[i] for i in range(0, t + 1))/4 >= -400
            model.minimum_energy_rule = Constraint(model.t, rule=minimum_energy_rule, doc='E min rule')

            def maximum_energy_rule(model, t):
                return sum(model.energy_charge[i]*efficiency - model.energy_discharge[i] for i in range(0, t + 1))/4 <= 400
            model.maximum_energy_rule = Constraint(model.t, rule=maximum_energy_rule, doc='E max rule')

#             def demand_charge_rule(model,t):
#                 return (model.net_load[t] + model.energy_charge[t] - model.energy_discharge[t]*efficiency) <= 570
#             model.demand_charge_rule = Constraint(model.t, rule=demand_charge_rule, doc='cannot exceed demand charge')

            def demand_charge_rule(model, t):
                return 20 * (model.energy_charge[t] - model.energy_discharge[t]*efficiency + model.net_load[t]-545) <= model.V
            model.demand_charge_rule = Constraint(model.t, rule=demand_charge_rule, doc='cannot exceed demand charge')

            def objective_rule(model):
                return sum((model.energy_charge[i] - model.energy_discharge[i]*efficiency + model.net_load[i])
                           * model.price_energy[i]  for i in model.t)/4 + model.V
            model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')
            
#             def objective_rule(model):
#                 return sum((model.energy_charge[i] - model.energy_discharge[i]*efficiency + model.net_load[i])
#                            * model.price_energy[i]  for i in model.t)/4 + 20 * norm([model.energy_charge[i] - model.energy_discharge[i]*efficiency + model.net_load[i]-545
#                                    for i in model.t],ord = numpy.inf)
#             model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

            results = opt.solve(model)

            result_charge = []
            result_discharge = []


            for i in model.energy_charge:
                result_charge.append(value(model.energy_charge[i]))
            for i in model.energy_discharge:
                result_discharge.append(value(model.energy_discharge[i]))
            optimalresult = pd.DataFrame(
                {'energy charge' : result_charge,
                 'energy discharge' : result_discharge}
            )


            print value(model.objective)
            print value(model.V)
            model.load(results) # Loading solution into results object

            if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
                print 'Got the optimal solution!'
            elif (results.solver.termination_condition == TerminationCondition.infeasible):
                print 'No infeasible solution!'
            else:
                # Something else is wrong
                print ('Solver Status:', results.solver.status)


    return optimalresult

In [3]:
# Load the price data
opt_input = pd.read_excel('/Users/wangdai/Desktop/challenge/data/optinput.xlsx', sheetname=0, header=0, index_col=None)



# # Optimization
myresult = Co_optimization(opt_input, 0.92)

SOEchange =numpy.cumsum([myresult['energy charge'][i] * 0.92 / 4 - myresult['energy charge'][i] / 4 for i in range(96)])
SOE = [SOEchange[i] + 400 for i in range(96)]

#print myresult.model.energy
myresult.to_csv('battery_schedule.csv',index_label='15min')

1915.51973891
246.505800117
	loading solutions stored in SolverResults objects.  By default, results
	from solvers are immediately loaded into the original model instance.
Got the optimal solution!


In [4]:
change = [myresult['energy charge'][i]*0.92 - myresult['energy discharge'][i] for i in range(96)]
SOEchange = numpy.cumsum(change)/4
SOE = [SOEchange[i] + 400 for i in range(96)]
output = pd.DataFrame({'col':SOE})
output.to_csv('batterySOE.csv')